<a href="https://colab.research.google.com/github/Dipanjana-Saha/PysparkCode/blob/main/pyspark_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('diapanjana_pyspark').getOrCreate()

3)’’’  
You are given an employee dataset containing information about employees and their managers. Each
employee has a manager_id that refers to another employee in the same table. Your task to use self-join
to find hierarchical relationships between employees, such as finding all employees under specific

In [ ]:
from pyspark.sql.functions import col
data = [(1,"Alice",None),(2,"Bob",1),(3,"Charlie",1),(4,"David",2),(5,"EVA",2),(6,"Frank",3),(7,"Grace",3)]
columns = ["employee_id","employee_name","manager_id"]

df=spark.createDataFrame(data,columns)
df.show()



+-----------+-------------+----------+
|employee_id|employee_name|manager_id|
+-----------+-------------+----------+
|          1|        Alice|      NULL|
|          2|          Bob|         1|
|          3|      Charlie|         1|
|          4|        David|         2|
|          5|          EVA|         2|
|          6|        Frank|         3|
|          7|        Grace|         3|
+-----------+-------------+----------+



In [ ]:
result=df.alias("emp").join(df.alias("mgr"), on = col("emp.manager_id")==col("mgr.employee_name"),how ="left")\
.select(col("mgr.employee_name").alias("Manager"),
    col("emp.employee_name").alias("Employee"))
result.printSchema()

root
 |-- Manager: string (nullable = true)
 |-- Employee: string (nullable = true)



In [ ]:

hierarchy_df = df.alias("emp").join(df.alias("mgr"),
                                col("emp.manager_id") == col("mgr.employee_id"), "left").select(
                                    col("mgr.employee_name").alias("Manager"),
                                    col("emp.employee_name").alias("Employee")
                                )
hierarchy_df.show()

+-------+--------+
|Manager|Employee|
+-------+--------+
|   NULL|   Alice|
|  Alice|     Bob|
|  Alice| Charlie|
|Charlie|   Frank|
|Charlie|   Grace|
|    Bob|   David|
|    Bob|     EVA|
+-------+--------+



4)’’’  
You are working as a data engineer and the company has a log system where timestamps are recorded
for every user action. your manager wants to know how much time each user spends between log in and
log out.  
  
The system generates logs with login_timestamp and logout_timestamp in hours, minutes & seconds.
The result should be formatted like "HH:mm:ss".’’’  

In [ ]:
from pyspark.sql.functions import *
data = [(1,"2025-01-31 08:00:00","2025-01-31 10:30:45"),(2,"2025-01-31 09:00:30","2025-01-31 \
12:15:10"),(3,"2025-01-31 07:45:00","2025-01-31 09:00:15")]

schema = ["user_id","login_timestamp","logout_timestamp"]

df=spark.createDataFrame(data,schema)
#df.show()

In [ ]:
df=df.withColumn("login_time",unix_timestamp("login_timestamp"))
df=df.withColumn("logout_time",unix_timestamp("logout_timestamp"))
df.show()


+-------+-------------------+-------------------+----------+-----------+
|user_id|    login_timestamp|   logout_timestamp|login_time|logout_time|
+-------+-------------------+-------------------+----------+-----------+
|      1|2025-01-31 08:00:00|2025-01-31 10:30:45|1738310400| 1738319445|
|      2|2025-01-31 09:00:30|2025-01-31 12:15:10|1738314030| 1738325710|
|      3|2025-01-31 07:45:00|2025-01-31 09:00:15|1738309500| 1738314015|
+-------+-------------------+-------------------+----------+-----------+



In [ ]:
df=df.withColumn("DURATION",col("logout_time")-col("login_time"))
df.show()

+-------+-------------------+-------------------+----------+-----------+--------+
|user_id|    login_timestamp|   logout_timestamp|login_time|logout_time|DURATION|
+-------+-------------------+-------------------+----------+-----------+--------+
|      1|2025-01-31 08:00:00|2025-01-31 10:30:45|1738310400| 1738319445|    9045|
|      2|2025-01-31 09:00:30|2025-01-31 12:15:10|1738314030| 1738325710|   11680|
|      3|2025-01-31 07:45:00|2025-01-31 09:00:15|1738309500| 1738314015|    4515|
+-------+-------------------+-------------------+----------+-----------+--------+



In [ ]:
df=df.withColumn("hours",(col("DURATION")/3600).cast("int"))
df=df.withColumn("minutes",((col("DURATION")%3600)/60).cast("int"))
df = df.withColumn("seconds",(col("DURATION")%60).cast("int"))
df.show()

+-------+-------------------+-------------------+----------+-----------+--------+-----+-------+-------+
|user_id|    login_timestamp|   logout_timestamp|login_time|logout_time|DURATION|hours|minutes|seconds|
+-------+-------------------+-------------------+----------+-----------+--------+-----+-------+-------+
|      1|2025-01-31 08:00:00|2025-01-31 10:30:45|1738310400| 1738319445|    9045|    2|     30|     45|
|      2|2025-01-31 09:00:30|2025-01-31 12:15:10|1738314030| 1738325710|   11680|    3|     14|     40|
|      3|2025-01-31 07:45:00|2025-01-31 09:00:15|1738309500| 1738314015|    4515|    1|     15|     15|
+-------+-------------------+-------------------+----------+-----------+--------+-----+-------+-------+



5)’’’  
Given a dataset of monthly sales records with salespeople names and their regions, calculate the month
with the highest sales for each region using window functions and the max() function. Ensure that the
result includes the region name, month, and sales value. Consider sales fluctuations, and the dataset
should contain multiple records for each region to test windowing correctly.

In [ ]:
data = [ ("Amit", "North", "Jan", 12000), ("Rajesh", "North", "Feb", 15000), ("Sunita", "North", "Mar",
11000), ("Meena", "South", "Jan", 17000), ("Ravi", "South", "Feb", 20000), ("Priya", "South", "Mar",
18000), ("Suresh", "East", "Jan", 10000), ("Vishal", "East", "Feb", 22000), ("Akash", "East", "Mar", 21000),
("Anjali", "West", "Jan", 15000), ("Deepak", "West", "Feb", 13000), ("Nidhi", "West", "Mar", 17000) ]

columns =  ["Salesperson", "Region", "Month", "Sales"]

df = spark.createDataFrame(data,columns)

df.show()

+-----------+------+-----+-----+
|Salesperson|Region|Month|Sales|
+-----------+------+-----+-----+
|       Amit| North|  Jan|12000|
|     Rajesh| North|  Feb|15000|
|     Sunita| North|  Mar|11000|
|      Meena| South|  Jan|17000|
|       Ravi| South|  Feb|20000|
|      Priya| South|  Mar|18000|
|     Suresh|  East|  Jan|10000|
|     Vishal|  East|  Feb|22000|
|      Akash|  East|  Mar|21000|
|     Anjali|  West|  Jan|15000|
|     Deepak|  West|  Feb|13000|
|      Nidhi|  West|  Mar|17000|
+-----------+------+-----+-----+



In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window
windowspex = Window.partitionBy("Region").orderBy(col("sales").desc())
ranked_df= df.withColumn("rank",row_number().over(windowspex))
ranked_df.show()


+-----------+------+-----+-----+----+
|Salesperson|Region|Month|Sales|rank|
+-----------+------+-----+-----+----+
|     Vishal|  East|  Feb|22000|   1|
|      Akash|  East|  Mar|21000|   2|
|     Suresh|  East|  Jan|10000|   3|
|     Rajesh| North|  Feb|15000|   1|
|       Amit| North|  Jan|12000|   2|
|     Sunita| North|  Mar|11000|   3|
|       Ravi| South|  Feb|20000|   1|
|      Priya| South|  Mar|18000|   2|
|      Meena| South|  Jan|17000|   3|
|      Nidhi|  West|  Mar|17000|   1|
|     Anjali|  West|  Jan|15000|   2|
|     Deepak|  West|  Feb|13000|   3|
+-----------+------+-----+-----+----+



In [ ]:
max_rank=ranked_df.filter(col("rank")==1).select("Region","Sales","Month")
max_rank.show()

+------+-----+-----+
|Region|Sales|Month|
+------+-----+-----+
|  East|22000|  Feb|
| North|15000|  Feb|
| South|20000|  Feb|
|  West|17000|  Mar|
+------+-----+-----+



)’’’In PySpark, StructType and StructField are used for defining schema in DataFrames. Consider a
scenario where you have employee data stored in a CSV file with the following structure:  
Define an explicit schema for this dataset using StructType and StructField.  
Load this data into a PySpark DataFrame using the defined schema.  
Extract the employees who belong to the "IT" department and have a salary greater than 70000.  
Split the Address column into two separate columns: City and State.  
Save the transformed data into a Parquet file.’’’  

In [ ]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType
from pyspark.sql.functions import *

schema = StructType([ StructField("Emp_ID", IntegerType(), True),
                    StructField("Name", StringType(), True),
                    StructField("Age", IntegerType(), True),
                    StructField("Salary", IntegerType(), True),
                    StructField("Department", StringType(), True),
                    StructField("Address", StringType(), True) ])

data = [ (101, "Rajesh", 30, 60000, "IT", "Mumbai-Maharashtra"), (102, "Priya", 28, 75000, "HR",
"Bengaluru-Karnataka"), (103, "Suresh", 35, 50000, "Finance", "Chennai-Tamil Nadu"), (104, "Anjali", 25,
80000, "IT", "Pune-Maharashtra"), (105, "Arjun", 40, 90000, "Management", "Hyderabad-Telangana") ]

df = spark.createDataFrame(data,schema)
df.show()


+------+------+---+------+----------+-------------------+
|Emp_ID|  Name|Age|Salary|Department|            Address|
+------+------+---+------+----------+-------------------+
|   101|Rajesh| 30| 60000|        IT| Mumbai-Maharashtra|
|   102| Priya| 28| 75000|        HR|Bengaluru-Karnataka|
|   103|Suresh| 35| 50000|   Finance| Chennai-Tamil Nadu|
|   104|Anjali| 25| 80000|        IT|   Pune-Maharashtra|
|   105| Arjun| 40| 90000|Management|Hyderabad-Telangana|
+------+------+---+------+----------+-------------------+



In [ ]:
df_filter= df.filter((col("Department")=='IT') & (col("Salary")>70000))
df_filter.show()

+------+------+---+------+----------+----------------+
|Emp_ID|  Name|Age|Salary|Department|         Address|
+------+------+---+------+----------+----------------+
|   104|Anjali| 25| 80000|        IT|Pune-Maharashtra|
+------+------+---+------+----------+----------------+



In [ ]:
transformed_df=df.withColumn("City",split(col("Address"),"-")[0])\
.withColumn("State",split(col("Address"),"-")[1])
transformed_df.show()

+------+------+---+------+----------+-------------------+---------+-----------+
|Emp_ID|  Name|Age|Salary|Department|            Address|     City|      State|
+------+------+---+------+----------+-------------------+---------+-----------+
|   101|Rajesh| 30| 60000|        IT| Mumbai-Maharashtra|   Mumbai|Maharashtra|
|   102| Priya| 28| 75000|        HR|Bengaluru-Karnataka|Bengaluru|  Karnataka|
|   103|Suresh| 35| 50000|   Finance| Chennai-Tamil Nadu|  Chennai| Tamil Nadu|
|   104|Anjali| 25| 80000|        IT|   Pune-Maharashtra|     Pune|Maharashtra|
|   105| Arjun| 40| 90000|Management|Hyderabad-Telangana|Hyderabad|  Telangana|
+------+------+---+------+----------+-------------------+---------+-----------+



In [ ]:
#save the data in parquest form
transformed_df.write.mode("overwrite").parquet("/downloads/data.parquet")

7)’’’  
You have a dataset of user activities in an e-commerce application, where each row represents an activity
 performed by a user. The dataset contains duplicate activity entries (based on user and activity type) and
you need to remove the duplicates. Furthermore, you want to keep only the most recent record for each
user, based on a timestamp column.  
  
Problem  
Remove duplicates based on user_id and activity_type.  
Keep only the most recent activity_timestamp for each user and activity type combination.

In [ ]:
data = [
    (1, "login", "2025-02-01 10:00:00"),
    (1, "view_product", "2025-02-01 10:05:00"),
    (1, "login", "2025-02-01 10:30:00"),
    (2, "purchase", "2025-02-01 11:00:00"),
    (2, "login", "2025-02-01 11:15:00"),
    (2, "view_product", "2025-02-01 11:30:00"),
    (3, "login", "2025-02-01 12:00:00"),
    (3, "login", "2025-02-01 12:05:00")
]

columns = ["user_id", "activity_type", "activity_timestamp"]

df = spark.createDataFrame(data, columns)
df.show()  # Use df.show() instead of display(df) in standard PySpark scripts
df.printSchema()


+-------+-------------+-------------------+
|user_id|activity_type| activity_timestamp|
+-------+-------------+-------------------+
|      1|        login|2025-02-01 10:00:00|
|      1| view_product|2025-02-01 10:05:00|
|      1|        login|2025-02-01 10:30:00|
|      2|     purchase|2025-02-01 11:00:00|
|      2|        login|2025-02-01 11:15:00|
|      2| view_product|2025-02-01 11:30:00|
|      3|        login|2025-02-01 12:00:00|
|      3|        login|2025-02-01 12:05:00|
+-------+-------------+-------------------+

root
 |-- user_id: long (nullable = true)
 |-- activity_type: string (nullable = true)
 |-- activity_timestamp: string (nullable = true)



In [ ]:
df_timestamp =  df.withColumn("activity_timestamp",col("activity_timestamp").cast("timestamp"))
df_timestamp.show()
df_timestamp.printSchema()

+-------+-------------+-------------------+
|user_id|activity_type| activity_timestamp|
+-------+-------------+-------------------+
|      1|        login|2025-02-01 10:00:00|
|      1| view_product|2025-02-01 10:05:00|
|      1|        login|2025-02-01 10:30:00|
|      2|     purchase|2025-02-01 11:00:00|
|      2|        login|2025-02-01 11:15:00|
|      2| view_product|2025-02-01 11:30:00|
|      3|        login|2025-02-01 12:00:00|
|      3|        login|2025-02-01 12:05:00|
+-------+-------------+-------------------+

root
 |-- user_id: long (nullable = true)
 |-- activity_type: string (nullable = true)
 |-- activity_timestamp: timestamp (nullable = true)



In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

windows_spec=Window.partitionBy("user_id","activity_type").orderBy(col("activity_timestamp").desc())
transformed_df=df_timestamp.withColumn("rank",row_number().over(windows_spec))
transformed_df.show()

+-------+-------------+-------------------+----+
|user_id|activity_type| activity_timestamp|rank|
+-------+-------------+-------------------+----+
|      1|        login|2025-02-01 10:30:00|   1|
|      1|        login|2025-02-01 10:00:00|   2|
|      1| view_product|2025-02-01 10:05:00|   1|
|      2|        login|2025-02-01 11:15:00|   1|
|      2|     purchase|2025-02-01 11:00:00|   1|
|      2| view_product|2025-02-01 11:30:00|   1|
|      3|        login|2025-02-01 12:05:00|   1|
|      3|        login|2025-02-01 12:00:00|   2|
+-------+-------------+-------------------+----+



In [ ]:
df_final=transformed_df.filter(col("rank")==1).drop("rank")
df_final.show()

+-------+-------------+-------------------+
|user_id|activity_type| activity_timestamp|
+-------+-------------+-------------------+
|      1|        login|2025-02-01 10:30:00|
|      1| view_product|2025-02-01 10:05:00|
|      2|        login|2025-02-01 11:15:00|
|      2|     purchase|2025-02-01 11:00:00|
|      2| view_product|2025-02-01 11:30:00|
|      3|        login|2025-02-01 12:05:00|
+-------+-------------+-------------------+



8)’’’You are working as a Data Engineer for a company. The sales team has provided you with a dataset
containing sales information. However, the data has some missing values that need to be addressed
before processing. You are required to perform the following tasks:  
1. Load the following sample dataset into a PySpark DataFrame:  
2. Perform the following operations:  
    a. Replace all NULL values in the Quantity column with 0.  
    b. Replace all NULL values in the Price column with the average price of the existing data.  
    c. Drop rows where the Product column is NULL.  
    d. Fill missing Sales_Date with a default value of ’2025-01-01’.  
    e. Drop rows where all columns are NULL.’’’

In [ ]:
from pyspark.sql.functions import *
data = [ (1, "Laptop", 10, 50000, "North", "2025-01-01"), (2, "Mobile", None, 15000, "South", None), (3,
"Tablet", 20, None, "West", "2025-01-03"), (4, "Desktop", 15, 30000, None, "2025-01-04"), (5, None,
None, None, "East", "2025-01-05") ]
columns = ["Sales_ID", "Product", "Quantity", "Price", "Region", "Sales_Date"]
df = spark.createDataFrame(data,columns)
df.show()
df.printSchema()

+--------+-------+--------+-----+------+----------+
|Sales_ID|Product|Quantity|Price|Region|Sales_Date|
+--------+-------+--------+-----+------+----------+
|       1| Laptop|      10|50000| North|2025-01-01|
|       2| Mobile|    NULL|15000| South|      NULL|
|       3| Tablet|      20| NULL|  West|2025-01-03|
|       4|Desktop|      15|30000|  NULL|2025-01-04|
|       5|   NULL|    NULL| NULL|  East|2025-01-05|
+--------+-------+--------+-----+------+----------+

root
 |-- Sales_ID: long (nullable = true)
 |-- Product: string (nullable = true)
 |-- Quantity: long (nullable = true)
 |-- Price: long (nullable = true)
 |-- Region: string (nullable = true)
 |-- Sales_Date: string (nullable = true)



1)You have been given a large dataset containing employee salary details. Your goal is to optimize a PySpark job that performs a groupBy operation while minimizing the shuffle.

Task:
Write a PySpark job to calculate the total salary per department.
Optimize the job to reduce shuffle while performing the groupBy operation.
Explain why your optimization reduces shuffle and improves performance.


Approach:
To minimize shuffle during a groupBy operation, we should:
Use repartition() efficiently to avoid unnecessary partitions.
Use reduceByKey() instead of groupByKey(), as it performs local aggregation before shuffling.
If working with a DataFrame, use partitionBy() while writing output.

In [ ]:
from pyspark.sql.functions import *
data = [ (101, "Rahul", "IT", 90000), (102, "Sita", "HR", 75000), (103, "Vikram", "IT", 85000), (104, "Priya", "HR", 72000), (105, "Anjali", "IT", 88000), (106, "Manish", "Sales", 67000), (107, "Neha", "Sales", 70000) ]

columns = ["emp_id", "name", "dept", "salary"]

df = spark.createDataFrame(data,columns)
df.show()

+------+------+-----+------+
|emp_id|  name| dept|salary|
+------+------+-----+------+
|   101| Rahul|   IT| 90000|
|   102|  Sita|   HR| 75000|
|   103|Vikram|   IT| 85000|
|   104| Priya|   HR| 72000|
|   105|Anjali|   IT| 88000|
|   106|Manish|Sales| 67000|
|   107|  Neha|Sales| 70000|
+------+------+-----+------+



In [ ]:
#optimzied approach
df1=df.repartition("dept").groupBy("dept").agg(sum("salary").alias("Total Salary"))
df1.show()

+-----+------------+
| dept|Total Salary|
+-----+------------+
|   HR|      147000|
|   IT|      263000|
|Sales|      137000|
+-----+------------+



In [ ]:
#anothre approach by using rdd
rdd=df.rdd.map(lambda x:(x[2],x[3]))
optimized_df=rdd.reduceByKey(lambda x,y:x+y).toDF(["dept","TotalSlary"])
optimized_df.show()

+-----+----------+
| dept|TotalSlary|
+-----+----------+
|   IT|    263000|
|   HR|    147000|
|Sales|    137000|
+-----+----------+



2)You are given a dataset containing sales transactions for different products. The dataset consists of columns for product_id, product_name, and sales_amount. Your task is to calculate the total sales for each product and filter out only the products with total sales greater than $1000.

In [ ]:
from pyspark.sql.functions import *
data = [(1, "Product A", 500), (1, "Product A", 600),(2, "Product B", 300), (3, "Product C", 1200),(3, "Product C", 800), (4, "Product D", 900)]

columns = ["product_id", "product_name", "sales_amount"]

df = spark.createDataFrame(data,columns)
df.show()

+----------+------------+------------+
|product_id|product_name|sales_amount|
+----------+------------+------------+
|         1|   Product A|         500|
|         1|   Product A|         600|
|         2|   Product B|         300|
|         3|   Product C|        1200|
|         3|   Product C|         800|
|         4|   Product D|         900|
+----------+------------+------------+



In [ ]:
df1=df.groupBy("product_name").agg(sum("sales_amount").alias("totalSales"))
df1.show()

+------------+----------+
|product_name|totalSales|
+------------+----------+
|   Product A|      1100|
|   Product B|       300|
|   Product C|      2000|
|   Product D|       900|
+------------+----------+



In [ ]:
result_df=df1.filter(col("totalSales")>1000)
result_df.show()

+------------+----------+
|product_name|totalSales|
+------------+----------+
|   Product A|      1100|
|   Product C|      2000|
+------------+----------+



3)You are working as a Data Engineer at a fintech company. Your team is working on integrating two datasets:
Customer Transactions Data (transactions_df) - Contains customer transactions with columns: customer_id, txn_id, amount, and txn_date.
Customer Profile Data (profile_df) - Contains customer information with columns: customer_id, name, age, and txn_id (latest transaction ID for reference).
The requirement is to merge these two DataFrames on customer_id while keeping track of:
Conflicting column names (txn_id) should be renamed properly.
If a customer exists in profile_df but not in transactions_df, the row should still be present with NULL values for transaction-related columns.
Your task is to write an optimized PySpark code to achieve this.

In [ ]:
transactions_data = [ (101, "T001", 500, "2024-08-10"), (102, "T002", 1200, "2024-08-09"), (103, "T003", 300, "2024-08-08"), (104, "T004", 450, "2024-08-07"), ]

profile_data = [ (101, "John", 30, "T001"), (102, "Emma", 27, "T005"), (103, "Alex", 35, "T003"), (105, "Sam", 40, "T006"), ]

columns1 = ["customer_id", "txn_id", "amount", "txn_date"]
columns2 = ["customer_id", "name", "age", "txn_id"]

transaction_df = spark.createDataFrame(transactions_data,columns1)
profile_df = spark.createDataFrame(profile_data,columns2)

transaction_df.show()
profile_df.show()


+-----------+------+------+----------+
|customer_id|txn_id|amount|  txn_date|
+-----------+------+------+----------+
|        101|  T001|   500|2024-08-10|
|        102|  T002|  1200|2024-08-09|
|        103|  T003|   300|2024-08-08|
|        104|  T004|   450|2024-08-07|
+-----------+------+------+----------+

+-----------+----+---+------+
|customer_id|name|age|txn_id|
+-----------+----+---+------+
|        101|John| 30|  T001|
|        102|Emma| 27|  T005|
|        103|Alex| 35|  T003|
|        105| Sam| 40|  T006|
+-----------+----+---+------+



In [ ]:
profile_df=profile_df.withColumnRenamed("txn_id","last_txn_id")
merge_df=profile_df.join(transaction_df,on="customer_id",how="outer")
merge_df.show()

+-----------+----+----+-----------+------+------+----------+
|customer_id|name| age|last_txn_id|txn_id|amount|  txn_date|
+-----------+----+----+-----------+------+------+----------+
|        101|John|  30|       T001|  T001|   500|2024-08-10|
|        102|Emma|  27|       T005|  T002|  1200|2024-08-09|
|        103|Alex|  35|       T003|  T003|   300|2024-08-08|
|        104|NULL|NULL|       NULL|  T004|   450|2024-08-07|
|        105| Sam|  40|       T006|  NULL|  NULL|      NULL|
+-----------+----+----+-----------+------+------+----------+



4)You are given a sales transaction dataset containing sales details. Your task is to write a PySpark transformation to:
Calculate the total sales per product.
Filter out products with total sales greater than $1000.

In [ ]:
from pyspark.sql.functions import *

data = [ (1, "P001", "Laptop", 2, 600), (2, "P002", "Phone", 3, 300), (3, "P001", "Laptop", 1, 600), (4, "P003", "Tablet", 5, 150), (5, "P002", "Phone", 1, 300) ]

columns = ["transaction_id", "product_id", "product_name", "quantity", "price_per_unit"]

df = spark.createDataFrame(data,columns)
df.show()

+--------------+----------+------------+--------+--------------+
|transaction_id|product_id|product_name|quantity|price_per_unit|
+--------------+----------+------------+--------+--------------+
|             1|      P001|      Laptop|       2|           600|
|             2|      P002|       Phone|       3|           300|
|             3|      P001|      Laptop|       1|           600|
|             4|      P003|      Tablet|       5|           150|
|             5|      P002|       Phone|       1|           300|
+--------------+----------+------------+--------+--------------+



In [ ]:
total=df.withColumn("totalPrice",col("quantity")*col("price_per_unit"))
total.show()

+--------------+----------+------------+--------+--------------+----------+
|transaction_id|product_id|product_name|quantity|price_per_unit|totalPrice|
+--------------+----------+------------+--------+--------------+----------+
|             1|      P001|      Laptop|       2|           600|      1200|
|             2|      P002|       Phone|       3|           300|       900|
|             3|      P001|      Laptop|       1|           600|       600|
|             4|      P003|      Tablet|       5|           150|       750|
|             5|      P002|       Phone|       1|           300|       300|
+--------------+----------+------------+--------+--------------+----------+



In [ ]:
total.groupBy("product_id","product_name").agg(sum("totalPrice").alias("final_total")).show()

+----------+------------+-----------+
|product_id|product_name|final_total|
+----------+------------+-----------+
|      P002|       Phone|       1200|
|      P001|      Laptop|       1800|
|      P003|      Tablet|        750|
+----------+------------+-----------+



5)You are working with large datasets in PySpark and need to join two DataFrames. However, one of the tables has highly skewed data, causing performance issues due to data shuffling. How would you optimize this join using salting techniques?
You are given the following sample datasets:
sales_df (Fact Table - Large Dataset, Highly Skewed on store_id)
Your task is to perform an optimized join between sales_df and store_df on store_id, ensuring that the skewness does not degrade performance.

6)Imagine you're analyzing the monthly sales performance of a company across different regions. You want to calculate:
The cumulative sales for each region over months.
The rank of each month based on sales within the same region.


In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.window import *

data = [ ("East", "Jan", 200), ("East", "Feb", 200), ("East", "Mar", 150), ("West", "Jan", 400), ("West", "Feb", 350), ("West", "Mar", 450) ]

columns = ["Region", "Month", "Sales"]

df = spark.createDataFrame(data,columns)
df.show()


+------+-----+-----+
|Region|Month|Sales|
+------+-----+-----+
|  East|  Jan|  200|
|  East|  Feb|  200|
|  East|  Mar|  150|
|  West|  Jan|  400|
|  West|  Feb|  350|
|  West|  Mar|  450|
+------+-----+-----+



In [ ]:
window_cum=Window.partitionBy("Region").orderBy("Month")

window_spec=Window.partitionBy("Region").orderBy(col("Sales").desc())

df=df.withColumn("cumulative",sum("Sales").over(window_cum))\
.withColumn("rank",rank().over(window_spec))
df.show()

+------+-----+-----+----------+----+
|Region|Month|Sales|cumulative|rank|
+------+-----+-----+----------+----+
|  East|  Feb|  200|       200|   1|
|  East|  Jan|  200|       400|   1|
|  East|  Mar|  150|       550|   3|
|  West|  Mar|  450|      1200|   1|
|  West|  Jan|  400|       750|   2|
|  West|  Feb|  350|       350|   3|
+------+-----+-----+----------+----+



In [ ]:
window_cum=Window.partitionBy("Region").orderBy("Month")

window_spec=Window.partitionBy("Region").orderBy(col("Sales").desc())

df=df.withColumn("cumulative",sum("Sales").over(window_cum))\
.withColumn("rank",rank().over(window_spec))
df.show()

+------+-----+-----+----------+----+
|Region|Month|Sales|cumulative|rank|
+------+-----+-----+----------+----+
|  East|  Feb|  200|       200|   1|
|  East|  Jan|  200|       400|   1|
|  East|  Mar|  150|       550|   3|
|  West|  Mar|  450|      1200|   1|
|  West|  Jan|  400|       750|   2|
|  West|  Feb|  350|       350|   3|
+------+-----+-----+----------+----+



7)You are given a large e-commerce transaction dataset stored in a partitioned format based on country. Your task is to count the distinct number of products purchased (product_id) for each customer_id in every country. The result should include the country, customer ID, and the distinct product count.

In [ ]:
from pyspark.sql.functions import *
data = [ ("USA", 101, "P001"), ("USA", 101, "P002"), ("USA", 101, "P001"), ("USA", 102, "P003"), ("USA", 102, "P003"), ("UK", 201, "P004"), ("UK", 201, "P005"), ("UK", 202, "P004"), ("UK", 202, "P005"), ("UK", 202, "P004") ]

columns = ["country", "customer_id", "product_id"]

df = spark.createDataFrame(data,columns)
df.show()

+-------+-----------+----------+
|country|customer_id|product_id|
+-------+-----------+----------+
|    USA|        101|      P001|
|    USA|        101|      P002|
|    USA|        101|      P001|
|    USA|        102|      P003|
|    USA|        102|      P003|
|     UK|        201|      P004|
|     UK|        201|      P005|
|     UK|        202|      P004|
|     UK|        202|      P005|
|     UK|        202|      P004|
+-------+-----------+----------+



In [ ]:
result_df=df.groupBy("country","customer_id").agg(countDistinct("product_id").alias("distincr_product")).orderBy("customer_id")
result_df.show()

+-------+-----------+----------------+
|country|customer_id|distincr_product|
+-------+-----------+----------------+
|    USA|        101|               2|
|    USA|        102|               1|
|     UK|        201|               2|
|     UK|        202|               2|
+-------+-----------+----------------+



8)

9)Write a PySpark program to calculate the highest salary in each department and sort the output in ascending order of department name.

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import *
data = [(1, "Ravi", "IT", 85000),(2, "Priya", "HR", 65000),(3, "Suresh", "IT", 92000),(4, "Neha", "Finance", 78000),
(5, "Anil", "HR", 72000),(6, "Divya", "Finance", 88000),(7, "Kiran", "IT", 88000)]

columns = ["emp_id", "emp_name", "department", "salary"]

df = spark.createDataFrame(data,columns)
df.show()

+------+--------+----------+------+
|emp_id|emp_name|department|salary|
+------+--------+----------+------+
|     1|    Ravi|        IT| 85000|
|     2|   Priya|        HR| 65000|
|     3|  Suresh|        IT| 92000|
|     4|    Neha|   Finance| 78000|
|     5|    Anil|        HR| 72000|
|     6|   Divya|   Finance| 88000|
|     7|   Kiran|        IT| 88000|
+------+--------+----------+------+



In [ ]:
window_spec=Window.partitionBy("department").orderBy(col("salary").desc())
rank_df = df.withColumn("rank",dense_rank().over(window_spec))
rank1=rank_df.filter(col("rank")==1).orderBy("department")
rank1.show()

+------+--------+----------+------+----+
|emp_id|emp_name|department|salary|rank|
+------+--------+----------+------+----+
|     6|   Divya|   Finance| 88000|   1|
|     5|    Anil|        HR| 72000|   1|
|     3|  Suresh|        IT| 92000|   1|
+------+--------+----------+------+----+



10)You are working as a Data Engineer at a retail company. The marketing team has provided a dataset of customer purchases to analyze the relationship between the amount spent on advertisements and the revenue generated. Using PySpark, compute the correlation between the "Ad_Spend" and "Revenue" columns to determine if there's a linear relationship.

In [ ]:
from pyspark.sql.types import *
schema = StructType([ StructField("Customer_ID", StringType(), True), StructField("Ad_Spend", IntegerType(), True), StructField("Revenue", IntegerType(), True) ])

data = [ ("C001", 2000, 25000), ("C002", 1500, 23000), ("C003", 3000, 40000), ("C004", 1200, 18000), ("C005", 2500, 30000) ]

df = spark.createDataFrame(data,schema)
display(df)

correl = df.stat.corr("Ad_Spend","Revenue")
display(correl)


DataFrame[Customer_ID: string, Ad_Spend: int, Revenue: int]

0.9704535552410215

You are given a dataset containing sales data for different stores across various months. Each row contains the store name, the month, and the sales amount. Your task is to calculate the cumulative sales for each store, considering the monthly sales, using PySpark.

You should also:
Filter out stores with sales lower than 1000 in any month.
Calculate the total sales for each store over all months.
Sort the results by the total sales in descending order.

In [ ]:
from pyspark.sql.types import *
data = [ ("Store A", "2024-01", 800), ("Store A", "2024-02", 1200), ("Store A", "2024-03", 900), ("Store B", "2024-01", 1500), ("Store B", "2024-02", 1600),
        ("Store B", "2024-03", 1400), ("Store C", "2024-01", 700), ("Store C", "2024-02", 1000), ("Store C", "2024-03", 800) ]

df=spark.createDataFrame(data,["Store", "Month", "Sales"])
df.show()

+-------+-------+-----+
|  Store|  Month|Sales|
+-------+-------+-----+
|Store A|2024-01|  800|
|Store A|2024-02| 1200|
|Store A|2024-03|  900|
|Store B|2024-01| 1500|
|Store B|2024-02| 1600|
|Store B|2024-03| 1400|
|Store C|2024-01|  700|
|Store C|2024-02| 1000|
|Store C|2024-03|  800|
+-------+-------+-----+



In [ ]:
df.filter(df.Sales < 1000).show()

+-------+-------+-----+
|  Store|  Month|Sales|
+-------+-------+-----+
|Store A|2024-01|  800|
|Store A|2024-03|  900|
|Store C|2024-01|  700|
|Store C|2024-03|  800|
+-------+-------+-----+



In [ ]:
#Calculate the total sales for each store over all months.
window

Q1)Problem Statement:

You are given a dataset that contains user activity logs, where each row represents a date a user logged into an app.
Write a PySpark program to identify gaps in login activity per user — i.e., the date ranges when the user was inactive.

✅ Input Columns:
user_id: String (e.g., "U001")
login_date: Date (e.g., "2024-01-01")

Assume:
Dates are not continuous.
Each user is expected to be active daily.

✅ Expected Output Columns:
user_id
inactive_from_date
inactive_to_date
Only output gaps of more than 1 day.

In [ ]:

from pyspark.sql.functions import col, lag, lead, to_date, datediff
from pyspark.sql.window import Window

data = [
    ("U001", "2024-01-01"),
    ("U001", "2024-01-02"),
    ("U001", "2024-01-05"),
    ("U001", "2024-01-07"),
    ("U002", "2024-01-03"),
    ("U002", "2024-01-07"),
]
column = ["user_id","login_date"]
df=spark.createDataFrame(data,column)
df=df.withColumn("login_date",col("login_date").cast("date"))
window_spec=Window.partitionBy("user_id").orderBy("login_date")
df=df.withColumn("previous_date",lag("login_date",1).over(window_spec))
df=df.withColumn("gap",datediff("login_date","previous_date"))
df=df.filter(col("gap")>1)
result=df.select("user_id",col("previous_date").alias("inactive_from_date"),col("login_date").alias("inactive_to_date"))
result.show()


+-------+------------------+----------------+
|user_id|inactive_from_date|inactive_to_date|
+-------+------------------+----------------+
|   U001|        2024-01-02|      2024-01-05|
|   U001|        2024-01-05|      2024-01-07|
|   U002|        2024-01-03|      2024-01-07|
+-------+------------------+----------------+



Scenario:
You’re working as a Data Engineer for a power distribution company in India. Customers are billed daily, but due to technical issues, some records are missing in the billing_logs table. Management wants to find out the missing billing dates for each customer.

Identify continuous date gaps in billing logs for each customer between their first and last billing date.

data = [
 ("C001", "2024-01-01"),
 ("C001", "2024-01-02"),
 ("C001", "2024-01-04"),
 ("C001", "2024-01-06"),
 ("C002", "2024-01-03"),
 ("C002", "2024-01-05"),
]

df = spark.createDataFrame(data, ["customer_id", "billing_date"])


Output :
+------------+------------+------------+
|customer_id |missing_from|missing_to |
+------------+------------+------------+
|C001 |2024-01-03 |2024-01-03 |
|C001 |2024-01-05 |2024-01-05 |
|C002 |2024-01-04 |2024-01-04 |
+------------+------------+------------+


In [ ]:

from pyspark.sql.functions import col, min, datediff
data = [
 ("U001", "2024-01-01"),
 ("U001", "2024-01-02"), # Day 1
 ("U001", "2024-01-08"), # Day 7
 ("U001", "2024-01-31"), # Day 30
 ("U002", "2024-01-01"),
 ("U002", "2024-01-03"),
 ("U003", "2024-01-02"),
 ("U003", "2024-01-03"), # Day 1
 ("U003", "2024-02-01"), # Day 30
]

# Create DataFrame
df = spark.createDataFrame(data, ["user_id", "login_date"])
df=df.withColumn("login_date",col("login_date").cast("date"))
signup_df = df.groupBy("user_id").agg(min("login_date").alias("signup_date"))
df=df.join(signup_df, on="user_id")

